Storing and Resampling Real Time Data (Part2)

In [1]:
import pandas as pd
import tpqoa
import time

In [2]:
class GetTickData(tpqoa.tpqoa):
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
        self.tick_count = 0
        self.target_ticks = None
        self.should_continue = True

    def on_success(self, time, bid, ask):
        self.tick_count += 1
        print(f"Tick {self.tick_count}/{self.target_ticks}: {time}, Bid: {bid}, Ask: {ask}")
        
        df = pd.DataFrame({
            "bid": bid, 
            "ask": ask, 
            "mid": (ask + bid)/2
        }, index=[pd.to_datetime(time)])
        
        self.tick_data = pd.concat([self.tick_data, df])
        self.data = self.tick_data.resample(self.bar_length, label="right").last().ffill()
        
        if self.tick_count >= self.target_ticks:
            self.should_continue = False

    def stream_data(self, instrument, stop=None, ret=False):
        self.tick_count = 0
        self.target_ticks = stop
        self.should_continue = True
        
        response = self.ctx.pricing.stream(
            self.account_id,
            snapshot=True,
            instruments=instrument
        )
        
        try:
            for msg_type, msg in response.parts():
                if not self.should_continue:
                    break
                    
                if msg_type == "pricing.ClientPrice":
                    self.on_success(msg.time,
                                  float(msg.bids[0].dict()['price']),
                                  float(msg.asks[0].dict()['price']))
                    
                    if not self.should_continue:
                        break
        finally:
            response.close()

In [3]:
td = GetTickData("oanda.cfg", "5s")

In [6]:
td.stream_data("EUR_USD", stop=20)

AttributeError: 'Response' object has no attribute 'close'

In [8]:
print("\nFinal resampled data:")
print(td.data)


Final resampled data:


AttributeError: 'GetTickData' object has no attribute 'data'